In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import json

import sys; sys.path.append("../src")
from cholec import CholecExample, CholecDataset, load_model, items_to_examples

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset = CholecDataset(split="test")
num_samples = 3
items = [dataset[i] for i in range(num_samples)]

In [3]:
gpt = load_model("gpt-4o")
claude = load_model("claude-3-5-sonnet-latest")
gemini = load_model("gemini-2.0-flash")

In [4]:
examples = items_to_examples(
    items,
    explanation_model="gpt-4o",
    evaluation_model="gpt-4o",
    baseline="vanilla",
    verbose=True
)

Time taken to get LLM answers: 37.964 seconds
Time taken to decompose into atomic claims: 3.515 seconds
Time taken to distill relevant features: 15.200 seconds
Time taken to calculate expert alignment scores: 6.802 seconds
Total time taken: 63.481 seconds


In [5]:
print(examples[0])

{'id': 'M2CCAI2016_video107_001.png', 'llm_explanation': "I'm unable to analyze or provide specific details about the image. However, I can offer general guidance on safe and unsafe regions during a laparoscopic cholecystectomy based on typical anatomical landmarks and surgical principles.\n\n### Safe Regions:\n1. **Triangle of Calot**: \n   - **Boundaries**: Cystic duct, common hepatic duct, and the inferior edge of the liver.\n   - **Importance**: Proper identification and dissection within this area are crucial for safely isolating the cystic duct and artery.\n\n2. **Gallbladder Fundus**:\n   - **Location**: The top part of the gallbladder.\n   - **Importance**: Often used as a starting point for dissection, moving towards the neck.\n\n### Unsafe Regions:\n1. **Common Bile Duct (CBD)**:\n   - **Location**: Runs parallel to the cystic duct but is larger and more posterior.\n   - **Risk**: Injury can lead to significant complications, including bile leaks and strictures.\n\n2. **Liver

In [6]:
examples = items_to_examples(
    items,
    explanation_model="claude-3-5-sonnet-latest",
    evaluation_model="gpt-4o",
    baseline="vanilla",
    verbose=True
)

Time taken to get LLM answers: 28.569 seconds
Time taken to decompose into atomic claims: 2.401 seconds
Time taken to distill relevant features: 8.936 seconds
Time taken to calculate expert alignment scores: 6.400 seconds
Total time taken: 46.305 seconds


In [7]:
examples[0].to_dict()

{'id': 'M2CCAI2016_video107_001.png',
 'llm_explanation': "Looking at this laparoscopic image of what appears to be a cholecystectomy in progress, I can provide the following analysis:\n\nAnatomical Landmarks and Safe/Unsafe Regions:\n- The image shows what appears to be the critical view of safety during a laparoscopic cholecystectomy\n- The fatty tissue and peritoneal covering has been dissected away\n- There appears to be two tubular structures visible (likely the cystic duct and cystic artery)\n- The liver edge is visible in the upper portion of the image\n- The dissection appears to be in Calot's triangle\n\nSafe Regions:\n- The anterior and posterior peritoneal layers have been cleared appropriately\n- The dissection plane appears to be staying close to the gallbladder wall\n- There is good visualization of the operative field\n\nPotentially Unsafe Regions:\n- The area deep to the visible tubular structures requires careful attention to avoid common bile duct injury\n- The juncti

In [8]:
examples = items_to_examples(
    items,
    explanation_model="gemini-2.0-flash",
    evaluation_model="gpt-4o",
    baseline="vanilla",
    verbose=True
)

Time taken to get LLM answers: 15.955 seconds
Time taken to decompose into atomic claims: 9.545 seconds
Time taken to distill relevant features: 91.067 seconds
Time taken to calculate expert alignment scores: 8.338 seconds
Total time taken: 124.906 seconds


In [9]:
examples[1].to_dict()

{'id': 'M2CCAI2016_video107_002.png',
 'llm_explanation': "Here's an analysis of the provided laparoscopic cholecystectomy image, focusing on safe and unsafe regions for dissection:\n\n**Anatomical Landmarks and Tissue Types:**\n\n*   **Gallbladder:** The gallbladder itself is visible, likely distended. Dissection directly on the gallbladder serosa is generally safe, but care must be taken to avoid perforation, especially if the wall is inflamed or thickened.\n*   **Liver:** The liver is seen superiorly. Dissection too close to the liver parenchyma can cause bleeding.\n*   **Cystic Duct:** The cystic duct is the tubular structure connecting the gallbladder to the common bile duct. This is a critical structure to identify and ligate safely.\n*   **Cystic Artery:** The cystic artery runs alongside the cystic duct. It must be identified, clipped, and divided.\n*   **Triangle of Calot (Hepatocystic Triangle):** This is the area bounded by the cystic duct, the common hepatic duct, and the i

In [ ]:
claude("Tell me something!")